In [1]:
import xml.etree.ElementTree as et
import re
import pandas as pd
import dateutil
import datetime

In [2]:
# parsing the xml file
xtree = et.parse("../data/Iphone_health/apple_health_export/exportar.xml")
xroot = xtree.getroot()

In [3]:
#Obtention of a dataframe from XML parsed

df_cols = ['type', 'sourceName', 'sourceVersion', 'device', 'unit', 'creationDate', 
           'startDate', 'endDate', 'value']
rows = []
for node in xroot:
    if node.tag == 'Record':
        type_data = node.attrib.get('type')
        sourceName = node.attrib.get('sourceName')
        sourceVersion = node.attrib.get('sourceVersion')
        deviceName = re.findall(r'name:(\w+)',node.attrib.get('device'))[0]
        unit = node.attrib.get('unit')
        creationDate = node.attrib.get('creationDate')
        startDate = node.attrib.get('startDate')
        endDate = node.attrib.get('endDate')
        value = node.attrib.get('value')
        rows.append({'type': type_data, 
                     'sourceName': sourceName,
                     'sourceVersion': sourceVersion,
                     'device': deviceName,
                     'unit': unit, 
                     'creationDate': creationDate, 
                     'startDate': startDate, 
                     'endDate': endDate, 
                     'value': value})

health_df = pd.DataFrame(rows, columns = df_cols)

In [4]:
# Regex to extract just the type of the type column
health_df['type'] = health_df['type'].apply(lambda string: re.findall(r'Identifier(\w+)', string)[0])

In [5]:
health_df['type'].value_counts()

StepCount                 34480
DistanceWalkingRunning    34470
FlightsClimbed             6156
HeadphoneAudioExposure     1640
SleepAnalysis              1248
Name: type, dtype: int64

In [6]:
health_df['creationDate'] = health_df['creationDate'].apply(dateutil.parser.parse)
health_df['startDate'] = health_df['startDate'].apply(dateutil.parser.parse)
health_df['endDate'] = health_df['endDate'].apply(dateutil.parser.parse)

In [7]:
# split the dataframe into the things we want
audio_exposure_df = health_df[health_df['type'] =='HeadphoneAudioExposure'][['sourceName', 'sourceVersion', 'device', 'unit', 'creationDate', 
           'startDate', 'endDate', 'value']].reset_index(drop=True)
sleep_df = health_df[health_df['type'] =='SleepAnalysis'][['sourceName', 'device', 'creationDate', 
           'startDate', 'endDate']].reset_index(drop=True)
exercise_df = health_df[health_df['type'].isin(['StepCount','DistanceWalkingRunning', 'FlightsClimbed'])].sort_values('startDate').reset_index(drop=True)

In [8]:
audio_exposure_df

,sourceName,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,iPhone de: Pol,13.3,AirPods,dBASPL,2019-12-27 13:02:08+02:00,2019-12-27 12:51:07+02:00,2019-12-27 13:06:25+02:00,69.0332
1,iPhone de: Pol,13.3,AirPods,dBASPL,2019-12-27 13:07:20+02:00,2019-12-27 13:06:25+02:00,2019-12-27 13:07:19+02:00,62.2285
2,iPhone de: Pol,13.3,AirPods,dBASPL,2019-12-27 21:08:06+02:00,2019-12-27 21:03:57+02:00,2019-12-27 21:08:02+02:00,70.9675
3,iPhone de: Pol,13.3,AirPods,dBASPL,2019-12-27 21:08:06+02:00,2019-12-27 21:08:02+02:00,2019-12-27 21:08:05+02:00,76.4604
4,iPhone de: Pol,13.3,AirPods,dBASPL,2019-12-28 18:43:52+02:00,2019-12-28 18:39:22+02:00,2019-12-28 18:43:45+02:00,66.7052
...,...,...,...,...,...,...,...,...
1635,iPhone de: Pol,13.3,AirPods,dBASPL,2020-07-15 15:18:20+02:00,2020-07-15 15:18:15+02:00,2020-07-15 15:18:18+02:00,46.1822
1636,iPhone de: Pol,13.3,EarPods,dBASPL,2020-07-21 20:55:57+02:00,2020-07-21 20:55:31+02:00,2020-07-21 20:55:34+02:00,24.9614
1637,iPhone de: Pol,13.3,EarPods,dBASPL,2020-07-21 23:25:25+02:00,2020-07-21 23:14:24+02:00,2020-07-21 23:19:00+02:00,52.9766
1638,iPhone de: Pol,13.3,EarPods,dBASPL,2020-07-21 23:39:02+02:00,2020-07-21 23:29:59+02:00,2020-07-21 23:30:02+02:00,45.9645


In [9]:
audio_exposure_df['value'] = audio_exposure_df['value'].astype(float)
exercise_df['value'] = exercise_df['value'].astype(float)

In [10]:
exercise_df.rename(columns={'creationDate': 'creationDatetime', 'startDate': 'startDatetime', 'endDate': 'endDatetime'},inplace=True)
sleep_df.rename(columns={'creationDate': 'creationDatetime', 'startDate': 'startDatetime', 'endDate': 'endDatetime'},inplace=True)

In [11]:
exercise_by_day = exercise_df[['type', 'creationDatetime', 'startDatetime', 'endDatetime', 'value']]
exercise_by_day['duration'] = exercise_by_day['endDatetime'] - exercise_by_day['startDatetime']
exercise_by_day['date'] = exercise_by_day['startDatetime'].dt.date

sleep_df = sleep_df[['creationDatetime', 'startDatetime', 'endDatetime']]
sleep_df['duration'] = sleep_df['endDatetime'] - sleep_df['startDatetime']
sleep_df['date'] = sleep_df['startDatetime'].dt.date

C:\Users\Pol\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Pol\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
exercise_by_day = exercise_by_day[['date', 'type', 'value', 'duration']].groupby(['type', 'date']).sum(numeric_only=False).reset_index()
sleep_by_day = sleep_df.groupby('date')['duration'].sum(numeric_only=False).reset_index()

In [15]:
exercise_by_day = exercise_by_day.pivot( index = 'date', columns = 'type').reset_index()

In [16]:
exercise_by_day.columns = ['date', 'distance', 'flightsClimbed', 'steps', 'durationDistance', 'durationFlights', 'durationSteps']

In [17]:
exercise_by_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   date              984 non-null    datetime64[ns] 
 1   distance          969 non-null    float64        
 2   flightsClimbed    890 non-null    float64        
 3   steps             969 non-null    float64        
 4   durationDistance  969 non-null    timedelta64[ns]
 5   durationFlights   890 non-null    timedelta64[ns]
 6   durationSteps     969 non-null    timedelta64[ns]
dtypes: datetime64[ns](1), float64(3), timedelta64[ns](3)
memory usage: 53.9 KB


In [18]:
exercise_by_day[exercise_by_day['date'].apply(lambda x : datetime.datetime.strftime(x, "%Y-%m-%d")) == '2018-10-21']

,date,distance,flightsClimbed,steps,durationDistance,durationFlights,durationSteps
351,2018-10-21,NaN,9.0,NaN,NaT,00:19:56,NaT


In [19]:
audio_exposure_df.to_csv('../data/audio_exposure.csv', sep=';', decimal=',')
sleep_df.to_csv('../data/sleep.csv', sep=';', decimal=',')
sleep_by_day.to_csv('../data/sleep_by_day.csv', sep=';', decimal=',')
exercise_df.to_csv('../data/exercise.csv', sep=';', decimal=',')
exercise_by_day.to_csv('../data/exercise_summary_day.csv', sep=';', decimal=',')